# Vector Index

HNSW index

## Payload index

+ keyword
+ integer
+ float
+ bool
+ geo
+ text

In [1]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
client = QdrantClient('localhost', port=6333)
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='hello_world_collection'), CollectionDescription(name='sparse_vector_collection'), CollectionDescription(name='multi_vector_collection'), CollectionDescription(name='star_charts'), CollectionDescription(name='wolt_food')])

In [2]:
# Create index
client.create_payload_index(
    collection_name='wolt_food',
    field_name='description',
    field_schema='keyword',
)

ResponseHandlingException: timed out

In [3]:
# Create index on nested fields
client.create_payload_index(
    collection_name='wolt_food',
    field_name='cafe.rating',
    field_schema='float',
)

ResponseHandlingException: timed out

In [4]:
# Full text index

client.create_payload_index(
    collection_name='wolt_food',
    field_name='description',
    field_schema=models.TextIndexParams(
        type='text',
        tokenizer=models.TokenizerType.WORD,
        min_token_len=3, 
        max_token_len=8,
        lowercase=True,
    )
)

ResponseHandlingException: timed out